In [1]:
!unzip -u '/content/drive/MyDrive/recommendation sys data/data.zip' -d '/content/drive/MyDrive/recommendation sys data'

Archive:  /content/drive/MyDrive/recommendation sys data/data.zip
   creating: /content/drive/MyDrive/recommendation sys data/data/
  inflating: /content/drive/MyDrive/recommendation sys data/data/test.csv  
  inflating: /content/drive/MyDrive/recommendation sys data/data/train.csv  


In [2]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095453 sha256=17f1dba3c32834f3c0849da3229df34e960834aadcae9129aaa5d0aa1721432e
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [1]:
import pandas as pd
import numpy as np

from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import SVDpp
from surprise import KNNBasic
from surprise.model_selection import cross_validate

train_path = '/content/drive/MyDrive/recommendation sys data/data/train.csv'
test_path = '/content/drive/MyDrive/recommendation sys data/data/test.csv'

df = pd.read_csv(train_path)
df.head()

,userId,itemId,rating,date
0,cgexjc,682978,4,2017-08-13 13:23:35
1,cgexjc,320898,4,2019-01-18 15:56:07
2,cgexjc,29028,4,2017-08-13 14:03:55
3,cgexjc,399148,5,2017-08-13 13:59:51
4,cgexjc,734055,4,2019-01-18 15:37:29


In [2]:
# We Must change the userId column with numeric
def FeatureEncoder(value):
    encoder = {}
    for val, key in enumerate(value):
        encoder[key] = val
    return encoder

dicts = FeatureEncoder(df['userId'])
df['userId'] = df['userId'].map(dicts)
df.tail()

,userId,itemId,rating,date
8612102,8612106,148210,1,2017-08-05 14:15:09
8612103,8612106,735535,4,2017-08-05 14:14:35
8612104,8612106,124242,3,2017-08-05 14:25:33
8612105,8612106,948393,4,2017-08-05 13:25:21
8612106,8612106,163356,4,2017-08-05 14:24:57


In [3]:
df["rating"].unique()

array([4, 5, 3, 2, 1])

In [4]:
reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(df[['userId', 'itemId', 'rating']], reader)

In [5]:
algo = SVD()
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=2, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 2 split(s).

                  Fold 1  Fold 2  Mean    Std     
RMSE (testset)    0.8175  0.8177  0.8176  0.0001  
MAE (testset)     0.6265  0.6266  0.6265  0.0000  
Fit time          73.28   71.41   72.34   0.94    
Test time         64.31   55.00   59.66   4.66    


{'test_rmse': array([0.81750591, 0.817721  ]),
 'test_mae': array([0.62651469, 0.6265559 ]),
 'fit_time': (73.2778046131134, 71.40705394744873),
 'test_time': (64.31310629844666, 55.00060033798218)}

In [9]:
algo.predict(uid=8612106, iid=735535, r_ui=4)

Prediction(uid=8612106, iid=735535, r_ui=4, est=3.1510956877167904, details={'was_impossible': False})

In [10]:
for user, item, rating in zip(df['userId'][-50:], df['itemId'][-50:], df['rating'][-50:]):
    pred = algo.predict(uid=user, iid=item, r_ui=rating)
    print(pred)

user: 8612106    item: 586286     r_ui = 4.00   est = 3.57   {'was_impossible': False}
user: 8612106    item: 552961     r_ui = 4.00   est = 2.97   {'was_impossible': False}
user: 8612106    item: 306641     r_ui = 2.00   est = 3.26   {'was_impossible': False}
user: 8612106    item: 903083     r_ui = 4.00   est = 3.51   {'was_impossible': False}
user: 8612106    item: 393399     r_ui = 2.00   est = 2.45   {'was_impossible': False}
user: 8612106    item: 346109     r_ui = 4.00   est = 2.89   {'was_impossible': False}
user: 8612106    item: 366777     r_ui = 4.00   est = 3.64   {'was_impossible': False}
user: 8612106    item: 436877     r_ui = 4.00   est = 3.94   {'was_impossible': False}
user: 8612106    item: 73426      r_ui = 3.00   est = 2.86   {'was_impossible': False}
user: 8612106    item: 411801     r_ui = 4.00   est = 3.47   {'was_impossible': False}
user: 8612106    item: 886236     r_ui = 4.00   est = 3.26   {'was_impossible': False}
user: 8612106    item: 977095     r_ui = 4.

In [15]:
df_res = pd.DataFrame(columns=['prediction'])
df_res

,prediction


In [17]:
df_test = pd.read_csv(test_path)
dicts = FeatureEncoder(df_test['userId'])
df_test['userId'] = df_test['userId'].map(dicts)
df_test.tail()

,userId,itemId,date
131353,131357,388934,2021-07-30 08:48:06
131354,131357,571188,2021-07-30 08:49:17
131355,131357,707682,2021-07-30 08:49:40
131356,131357,44761,2021-07-30 08:48:55
131357,131357,699794,2021-07-30 14:21:08


In [22]:
rating_pred = []
for user, item in zip(df_test['userId'], df_test['itemId']):
    pred = algo.predict(uid=user, iid=item)
    rating_pred.append(pred[-2])

df_res['prediction'] = rating_pred

In [23]:
df_res

,prediction
0,3.880514
1,3.846657
2,3.726604
3,3.287340
4,3.465333
...,...
131353,4.060595
131354,3.491818
131355,3.506015
131356,2.926158


In [29]:
df_res.to_csv('output.csv')
!cp output.csv "drive/My Drive/"